In [ ]:
using JuMP
try import Mosek; using MosekTools; catch err; println("MOSEK not installed"); end
try import SCS;   catch err; println("SCS not installed"); end
try import SDPA;  catch err; println("SDPA not installed"); end
try import Clarabel; catch err; println("Clarabel not installed"); end

# Tasks:
# Write LP. Write in vectorized notation.
# Extract feasible point, optimal value, solver status, duality gap.
# Formulate dual (by hand). Solve.

# Next week:
# Formulate LP with *no* solution
# Extract infeasibility certificate.
# LP's for weighted matching, stable set,.. problems

MOSEK not installed
Clarabel not installed


### PRIMAL

In [3]:
model = Model(SCS.Optimizer)

A JuMP Model
├ solver: SCS
├ objective_sense: FEASIBILITY_SENSE
├ num_variables: 0
├ num_constraints: 0
└ Names registered in the model: none

In [4]:
@variable(model, x[1:4])  # Note: you can directly add some constraints in the variable declaration, e.g. @variable(model, x[1:4] ≥ 0) 

4-element Vector{VariableRef}:
 x[1]
 x[2]
 x[3]
 x[4]

In [17]:
#@constraint(model,  [i=1:4], x[i] ≤ 1)
A = [1 2 3 4; 5 6 7 8]
b = [1; 2]
@constraint(model, A*x ≤ b)
c = [1; 2; 3; 4]
@objective(model, Max, c'*x);
print(model)

In [6]:
optimize!(model)

------------------------------------------------------------------
	       SCS v3.2.8 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 4, constraints m: 2
cones: 	  l: linear vars: 2
settings: eps_abs: 1.0e-004, eps_rel: 1.0e-004, eps_infeas: 1.0e-007
	  alpha: 1.50, scale: 1.00e-001, adaptive_scale: 1
	  max_iters: 100000, normalize: 1, rho_x: 1.00e-006
	  acceleration_lookback: 10, acceleration_interval: 10
	  compiled with openmp parallelization enabled
lin-sys:  sparse-direct-amd-qdldl
	  nnz(A): 8, nnz(P): 0
------------------------------------------------------------------
 iter | pri res | dua res |   gap   |   obj   |  scale  | time (s)
------------------------------------------------------------------
     0|6.22e+001 4.00e+000 6.12e+001 -3.26e+001 1.00e-001 3.11e-003 
    25|1.74e-005 8.02e-008 1.74e-005 -1.00e+000 1.00e-001 3.51e-003 
-------------------------

In [7]:
x_o = value.(x)

4-element Vector{Float64}:
 -0.08879632338479829
  0.06516832533956753
  0.12442729410368644
  0.14629009986900038

In [8]:
α = objective_value(model)

0.9999826090813977

In [9]:
solution_summary(model)

solution_summary(; result = 1, verbose = false)
├ solver_name          : SCS
├ Termination
│ ├ termination_status : OPTIMAL
│ ├ result_count       : 1
│ └ raw_status         : solved
├ Solution (result = 1)
│ ├ primal_status        : FEASIBLE_POINT
│ ├ dual_status          : FEASIBLE_POINT
│ ├ objective_value      : 9.99983e-01
│ └ dual_objective_value : 1.00000e+00
└ Work counters
  └ solve_time (sec)   : 3.51110e-03

#### Note: solution summary gave _minus_ the objective value, at least for SCS solver. Seems to be an SCS internal convention.

### DUAL - by hand

In [10]:
dual_model = Model(SCS.Optimizer)
@variable(dual_model, y[1:2])
@objective(dual_model, Min, b'y)
@constraint(dual_model, A'*y == c)
@constraint(dual_model, y .≥ 0)
print(dual_model)
optimize!(dual_model)

------------------------------------------------------------------
	       SCS v3.2.8 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 2, constraints m: 6
cones: 	  z: primal zero / dual free vars: 4
	  l: linear vars: 2
settings: eps_abs: 1.0e-004, eps_rel: 1.0e-004, eps_infeas: 1.0e-007
	  alpha: 1.50, scale: 1.00e-001, adaptive_scale: 1
	  max_iters: 100000, normalize: 1, rho_x: 1.00e-006
	  acceleration_lookback: 10, acceleration_interval: 10
	  compiled with openmp parallelization enabled
lin-sys:  sparse-direct-amd-qdldl
	  nnz(A): 10, nnz(P): 0
------------------------------------------------------------------
 iter | pri res | dua res |   gap   |   obj   |  scale  | time (s)
------------------------------------------------------------------
     0|3.95e+000 2.01e+000 4.80e-001 2.23e+000 1.00e-001 6.38e-004 
    25|1.14e-007 5.98e-008 8.01e-009 1.00e+000 1.00e-001

In [11]:
y_o = value.(y)

2-element Vector{Float64}:
 0.9999999621566832
 4.6741521220820975e-9

In [12]:
β = objective_value(dual_model)

0.9999999715049873

In [13]:
solution_summary(dual_model)

solution_summary(; result = 1, verbose = false)
├ solver_name          : SCS
├ Termination
│ ├ termination_status : OPTIMAL
│ ├ result_count       : 1
│ └ raw_status         : solved
├ Solution (result = 1)
│ ├ primal_status        : FEASIBLE_POINT
│ ├ dual_status          : FEASIBLE_POINT
│ ├ objective_value      : 1.00000e+00
│ └ dual_objective_value : 1.00000e+00
└ Work counters
  └ solve_time (sec)   : 1.06740e-03

In [14]:
# duality gap
β - α

1.7362423589628584e-5

### DUAL - with package

In [15]:
using Dualization
model2 = Model(Mosek.Optimizer)
@variable(model2, x[1:2] ≤ 1)
dual_model = dualize(model)
print(dual_model)

LoadError: ArgumentError: Package Dualization not found in current path.
- Run `import Pkg; Pkg.add("Dualization")` to install the Dualization package.